In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from scipy.signal import savgol_filter
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
from tensorflow.keras.layers import Layer, GlobalAveragePooling2D, GlobalMaxPooling2D, Dense, Reshape, Multiply, Conv2D, Concatenate, Activation, Add, Input, Flatten, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications import InceptionResNetV2


def read_multiclass_data(base_path):
    data = {}
    all_images, all_labels = [], []
    label_map = {}
    for zoom_level in os.listdir(base_path):
        zoom_path = os.path.join(base_path, zoom_level)
        if os.path.isdir(zoom_path):
            data[zoom_level] = {}
            for class_index, class_name in enumerate(os.listdir(zoom_path)):
                class_path = os.path.join(zoom_path, class_name)
                if os.path.isdir(class_path):
                    label_map[class_name] = class_index
                    image_paths = [
                        os.path.join(class_path, f)
                        for f in os.listdir(class_path)
                        if f.endswith(('.png', '.jpg', '.jpeg'))
                    ]
                    data[zoom_level][class_name] = image_paths
                    all_images.extend(image_paths)
                    all_labels.extend([class_index] * len(image_paths))
    return data, all_images, all_labels, label_map


def balance_dataset(image_paths, labels):
    class_distribution = defaultdict(int)
    for label in labels:
        class_distribution[label] += 1
    mean_count = np.mean(list(class_distribution.values()))

    balanced_image_paths, balanced_labels = [], []
    for class_id in class_distribution:
        class_indices = [i for i, label in enumerate(labels) if label == class_id]
        class_images = [image_paths[i] for i in class_indices]
        if class_distribution[class_id] < mean_count:
            oversampled_images = np.random.choice(class_images, size=int(mean_count - class_distribution[class_id]), replace=True)
            balanced_image_paths.extend(class_images + list(oversampled_images))
            balanced_labels.extend([class_id] * len(class_images + list(oversampled_images)))
        else:
            undersampled_images = np.random.choice(class_images, size=int(mean_count), replace=False)
            balanced_image_paths.extend(list(undersampled_images))
            balanced_labels.extend([class_id] * len(undersampled_images))
    return balanced_image_paths, balanced_labels


class CBAM(Layer):
    def __init__(self, reduction_ratio=16):
        super(CBAM, self).__init__()
        self.reduction_ratio = reduction_ratio

    def build(self, input_shape):
        channel = input_shape[-1]
        self.shared_dense_one = Dense(channel // self.reduction_ratio, activation='relu', kernel_initializer='he_normal', use_bias=True, bias_initializer='zeros')
        self.shared_dense_two = Dense(channel, kernel_initializer='he_normal', use_bias=True, bias_initializer='zeros')
        self.conv2d = Conv2D(filters=1, kernel_size=7, strides=1, padding='same', activation='sigmoid', kernel_initializer='he_normal', use_bias=False)

    def call(self, input_feature):

        avg_pool = GlobalAveragePooling2D()(input_feature)
        avg_pool = Reshape((1, 1, input_feature.shape[-1]))(avg_pool)
        avg_pool = self.shared_dense_one(avg_pool)
        avg_pool = self.shared_dense_two(avg_pool)
        max_pool = GlobalMaxPooling2D()(input_feature)
        max_pool = Reshape((1, 1, input_feature.shape[-1]))(max_pool)
        max_pool = self.shared_dense_one(max_pool)
        max_pool = self.shared_dense_two(max_pool)
        channel_attention = Activation('sigmoid')(Add()([avg_pool, max_pool]))
        channel_refined = Multiply()([input_feature, channel_attention])
        # Spatial Attention
        avg_pool = tf.reduce_mean(channel_refined, axis=-1, keepdims=True)
        max_pool = tf.reduce_max(channel_refined, axis=-1, keepdims=True)
        concat = Concatenate(axis=-1)([avg_pool, max_pool])
        spatial_attention = self.conv2d(concat)
        refined_feature = Multiply()([channel_refined, spatial_attention])
        return refined_feature


def build_model_with_cbam(input_shape=(224, 224, 3), num_classes=8):
    base_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = True
    inputs = Input(shape=input_shape)
    x = base_model(inputs, training=False)
    x = CBAM()(x)
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    return Model(inputs, outputs)


def apply_clahe_to_color_image(image):
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)
    ca = clahe.apply(a)
    cb = clahe.apply(b)
    lab = cv2.merge((cl, ca, cb))
    return cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

def preprocess_image(image_path, target_size, augment=False, apply_clahe=False):
    image = cv2.imread(image_path.decode('utf-8'))
    if apply_clahe:
        image = apply_clahe_to_color_image(image)
    image = cv2.resize(image, target_size)
    image = image.astype('float32') / 255.0
    if augment:
        image = tf.image.random_flip_left_right(image)
        image = tf.image.random_flip_up_down(image)
    return image

def preprocess_image_wrapper(image_path, label, target_size, augment, apply_clahe):
    def _preprocess(path):
        return preprocess_image(path, target_size, augment, apply_clahe)
    image = tf.numpy_function(_preprocess, inp=[image_path], Tout=tf.float32)
    image.set_shape((*target_size, 3))
    return image, label


def create_dataset(image_paths, labels, batch_size=32, target_size=(224, 224), augment=False, apply_clahe=False):
    labels = to_categorical(labels, num_classes=len(set(labels)))
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    dataset = dataset.map(
        lambda x, y: preprocess_image_wrapper(x, y, target_size, augment, apply_clahe),
        num_parallel_calls=tf.data.AUTOTUNE,
    )
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset


def main():
    base_path = "/kaggle/input/breakhis-breast-cancer-histopathological-dataset/BreakHis - Breast Cancer Histopathological Database/dataset_cancer_v1/dataset_cancer_v1/classificacao_multiclasse"


    multiclass_data, all_images, all_labels, label_map = read_multiclass_data(base_path)
    balanced_paths, balanced_labels = balance_dataset(all_images, all_labels)


    train_paths, temp_paths, train_labels, temp_labels = train_test_split(
        balanced_paths, balanced_labels, test_size=0.3, random_state=42, stratify=balanced_labels)
    val_paths, test_paths, val_labels, test_labels = train_test_split(
        temp_paths, temp_labels, test_size=0.5, random_state=42, stratify=temp_labels)


    train_dataset = create_dataset(train_paths, train_labels, augment=True, apply_clahe=True)
    val_dataset = create_dataset(val_paths, val_labels, apply_clahe=True)
    test_dataset = create_dataset(test_paths, test_labels, apply_clahe=True)


    model = build_model_with_cbam(num_classes=len(label_map))
    model.compile(
        optimizer=Adam(learning_rate=1e-3),
        loss='categorical_crossentropy',
        metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
    )


    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-5)


    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=50,
        callbacks=[early_stopping, lr_scheduler]
    )


    y_pred = model.predict(test_dataset)
    y_true = np.array(test_labels)
    y_true = y_true.argmax(axis=1) if y_true.ndim > 1 else y_true
    y_pred = y_pred.argmax(axis=1)
    print("Classification Report:")
    print(classification_report(y_true, y_pred, target_names=label_map.keys()))


    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    tflite_model = converter.convert()


    with open('/kaggle/working/model.tflite', 'wb') as f:
        f.write(tflite_model)
    print("TFLite model saved to /kaggle/working/model.tflite")

if __name__ == "__main__":
    main()

2025-06-23 08:28:14.099617: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750667294.612635      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750667294.746040      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
I0000 00:00:1750667316.778379      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1750667316.779158      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability:

219055592/219055592 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/50


I0000 00:00:1750667464.861779     112 service.cc:148] XLA service 0x7a46a0003470 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1750667464.863604     112 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1750667464.863624     112 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1750667479.176920     112 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1750667533.958764     112 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


173/173 ━━━━━━━━━━━━━━━━━━━━ 411s 1s/step - accuracy: 0.1328 - loss: 2.6301 - precision: 0.1660 - recall: 0.0375 - val_accuracy: 0.1594 - val_loss: 26.8253 - val_precision: 0.1537 - val_recall: 0.0784 - learning_rate: 0.0010
Epoch 2/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 114s 660ms/step - accuracy: 0.1330 - loss: 2.3507 - precision: 0.1885 - recall: 0.0180 - val_accuracy: 0.1248 - val_loss: 2.0982 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 0.0010
Epoch 3/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 114s 659ms/step - accuracy: 0.1517 - loss: 2.2339 - precision: 0.2270 - recall: 0.0121 - val_accuracy: 0.1636 - val_loss: 2.0618 - val_precision: 0.6000 - val_recall: 0.0051 - learning_rate: 0.0010
Epoch 4/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 114s 659ms/step - accuracy: 0.2521 - loss: 1.9375 - precision: 0.4836 - recall: 0.0472 - val_accuracy: 0.3170 - val_loss: 2.4843 - val_precision: 0.5035 - val_recall: 0.0607 - learning_rate: 0.0010
Epoch 5/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 114s 657ms/st

W0000 00:00:1750672868.601843      35 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1750672868.601884      35 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
I0000 00:00:1750672869.074114      35 mlir_graph_optimization_pass.cc:401] MLIR V1 optimization pass is not enabled


TFLite model saved to /kaggle/working/model.tflite
